In [242]:
import subprocess
import zipfile
import os
import numpy as np
import pandas as pd
from sklearn.calibration import LabelEncoder

In [ ]:
subprocess.run(['wget', 'http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/MachineLearningCSV.zip', '-O', 'CIC_IDS_2017.zip'], check=True) 

with zipfile.ZipFile("CIC_IDS_2017.zip", 'r') as zip_ref:
    zip_ref.extractall("./data")
    
df_list = []
for file in os.listdir('./data/MachineLearningCVE/'):
  df_aux = pd.read_csv(f'./data/MachineLearningCVE/{file}')
  df_list.append(df_aux)

In [244]:
df = pd.concat(df_list, ignore_index=True)

In [245]:
df.columns = df.columns.str.strip()

In [ ]:
features = df.dtypes[df.dtypes != 'object'].index
df[features] = df[features].apply(
    lambda x: (x - x.mean()) / (x.std()))

In [247]:
df = df.fillna(0)

labelencoder = LabelEncoder()
df.iloc[:, -1] = labelencoder.fit_transform(df.iloc[:, -1])

In [ ]:
# After fitting the labelencoder
original_labels = labelencoder.classes_

# Create a mapping from original label to encoded value
label_mapping = {label: idx for idx, label in enumerate(original_labels)}

print("Label Mapping:")
for label, code in label_mapping.items():
    print(f"{label}: {code}")


In [ ]:
df.Label.value_counts()

In [250]:
df_minor = df[(df['Label'] == 9) | (df['Label'] == 8) | (df['Label'] == 13) | (df['Label'] == 14) | (df['Label'] == 12) | (df['Label'] == 1) ]
df_major = df.drop(df_minor.index)

In [251]:
X = df_major.drop(['Label'],axis=1)
y = df_major.iloc[:, -1].values.reshape(-1,1)
y=np.ravel(y)

In [ ]:
from sklearn.cluster import MiniBatchKMeans
# KMeans clustering
kmeans = MiniBatchKMeans(n_clusters=1000, random_state=0).fit(X)
df_major['klabel'] = kmeans.labels_

In [253]:
cols = list(df_major)
cols.insert(78, cols.pop(cols.index('Label')))
df_major = df_major.loc[:, cols]

In [254]:
def benignSampling(group):
    return group.sample(frac=0.02)

In [255]:
def typicalSampling(group):
    return group.sample(frac=0.0045)

In [256]:
df_label_0 = df_major[df_major['Label'] == 0]
df_other_labels = df_major[df_major['Label'] != 0]


result_0 = df_label_0.groupby(
    'klabel', group_keys=False
).apply(typicalSampling)

result_1 = df_other_labels.groupby(
    'klabel', group_keys=False
).apply(benignSampling)


In [257]:
result = pd.concat([result_0, result_1, df_minor], ignore_index=True)
result = result.drop(['klabel'], axis=1)

In [ ]:
# Save in the same directory
result.to_csv('CICIDS2017_sample_km.csv', index=False)
print("Final sampled dataset saved to CICIDS2018_sample_km.csv")

In [259]:
df = pd.read_csv('CICIDS2017_sample_km.csv')

In [ ]:
df.Label.value_counts()